<a href="https://colab.research.google.com/github/FrankPerera04/DSPL_Individual_CW/blob/main/Setting_up_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instructions for Usage**
1. Install in packages
2. Write the App
3. Start streamlit and tunnel
4. Get the ipv4 code
5. Run and get the Tunnel link
6. Enter the ipv4 code into the tunnel website to access streamlit

In [ ]:
!pip install streamlit -q
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
changed 22 packages in 2s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [ ]:
# Write the App
%%writefile app.py
"This app Streamlit App is running in Colab!"

In [ ]:
# Start Streamlit and tunnel in the background
import threading
import time

def run_app():
    get_ipython().system('streamlit run app.py &')

threading.Thread(target=run_app).start()
time.sleep(10)  # wait for Streamlit to start




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.70.191.172:8501



In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.70.191.172


In [ ]:
# Create the tunnel
get_ipython().system('npx localtunnel --port 8501')

⠙⠹⠸⠼⠴⠦⠧your url is: https://thirty-ghosts-thank.loca.lt
  Stopping...
^C
